## CJ 올리브영 스킨케어 상품군 리뷰 데이터 크롤링 

### import

In [59]:
# !pip install selenium
# !pip install tqdm
# !pip install beautifulsoup4
# !pip install pyautogui
# !pip install pandas
# !pip install webdriver-manager

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/19/5a/a16653bfce685c9832217d377f52065351eeac9862e44e2996cd81f3bb4d/webdriver_manager-4.0.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/9c/71/bf12b8e0d6e1d84ed29c3e16ea1efc47ae96487bde823130d12139c434a0/charset_normalizer-3.2.0-cp38-cp38-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 7.9 MB/s eta 0:00:00


In [14]:
import time
import os
import pyautogui as pag
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [33]:
# table 정도 한번에 불러오는 html_table_paser의 make2d 기능 test

from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import pandas as pd

In [69]:
from selenium import webdriver # webdriver를 이용해 해당 브라우저를 열기 위해
from selenium.webdriver import ActionChains # 일련의 작업들을(ex.아이디 입력, 비밀번호 입력, 로그인 버튼 클릭...) 연속적으로 실행할 수 있게 하기 위해
from selenium.webdriver.common.keys import Keys # 키보드 입력을 할 수 있게 하기 위해
from selenium.webdriver.common.by import By # html요소 탐색을 할 수 있게 하기 위해
from selenium.webdriver.support.ui import WebDriverWait # 브라우저의 응답을 기다릴 수 있게 하기 위해
from selenium.webdriver.support import expected_conditions as EC # html요소의 상태를 체크할 수 있게 하기 위해
# 이 외에도 필요한 모듈이 있다면 따로 호출해준다.

In [72]:
# 올리브영 상품 페이지 접근 
url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000186966&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%ED%86%A0%EB%84%88/%EB%A1%9C%EC%85%98/%EC%98%AC%EC%9D%B8%EC%9B%90_%EC%A0%84%EC%B2%B4__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=1"
driver = webdriver.Chrome()
driver.get(url)

In [74]:
# 리뷰 페이지 접근 
review = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').click()

In [75]:
# 리뷰 페이지 전환
page = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a:nth-child(2)')
page.click()

#### feature name 
- name :  상품명
- brand : 브랜드명
- price :  정가
- sale_price : 할인가
- date : 게시 날짜
- rate : 평점
- name : 닉네임
- skin_type : 고객타입 ex. 복합성, 민감성, 잡티
- purchase_type : 구매 타입 ex. 재구매, 한달이상사용
- select_1 : 피부타입   ex. 복합성에 좋아요
- select_2 : 피부고민  ex. 진정에 좋아요 
- select_3 : 자극도 ex. 자극없이 순해요 
- txt : 리뷰 텍스트

### 1. 스킨케어-토너/로션/올인원-스킨/토너 제품군 크롤링 test

#### 1-1 상품 기본 정보 가져오기 (상품명, 브랜드명, 정가, 할인가)

In [77]:
# 상품명, 브랜드명, 정가, 할인가 크롤링
name = driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name")
brand = driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_brand")
price = driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike")
sale_price = driver.find_element(By.CSS_SELECTOR,"#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong")
name.text, brand.text, price.text, sale_price.text

('[7월올영픽/대용량] 라운드랩 1025 독도 토너 500ml+200ml 기획(+소나무 클렌저 10ml 증정)',
 '라운드랩',
 '45,000',
 '29,800')

In [78]:
# 텍스트 추출 
# price 정보는 str 형태이기 때문에 콤마 제거하고 int 형태로 변환 
name = name.text
brand = brand.text
price = int(price.text.replace(",", ""))
sale_price = int(sale_price.text.replace(",", ""))
name, brand, price, sale_price

('[7월올영픽/대용량] 라운드랩 1025 독도 토너 500ml+200ml 기획(+소나무 클렌저 10ml 증정)',
 '라운드랩',
 45000,
 29800)

### 1-2 리뷰 데이터 가져오기 

- date : 게시 날짜
- rate : 평점
- id : 닉네임
- skin_type : 고객타입 ex. 복합성, 민감성, 잡티
- purchase_type : 구매 타입 ex. 재구매, 한달이상사용
- select_1 : 피부타입   ex. 복합성에 좋아요
- select_2 : 피부고민  ex. 진정에 좋아요 
- select_3 : 자극도 ex. 자극없이 순해요 
- txt : 리뷰 텍스트

In [42]:
date = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.score_area > span.date')
rate = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.score_area > span.review_point > span')
id = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.info > div > p.info_user > a.id')
skin_type = driver.find_element_by_css_selector('#gdasList > li:nth-child(2) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span')
# 한달이상사용 
# purchase_type = driver.find_element_by_css_selector('#gdasList > li:nth-child(4) > div.info > div > div > div > a')
# 재구매
purchase_type_2 = driver.find_element_by_css_selector('#gdasList > li:nth-child(9) > div.info > div > div > div:nth-child(1) > a')
select_1_title = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(1) > dt > span')
select_1_content = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span')
select_2_title = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(2) > dt > span')
select_2_content = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span')
select_3_title = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(3) > dt > span')
select_3_content = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span')
txt = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner')

In [43]:
date = date.text
rate = rate.text
id = id.text
skin_type = skin_type.text
# purchase_type = purchase_type.text
select_1_title = select_1_title.text
select_1_content = select_1_content.text
select_2_title = select_2_title.text
select_2_content = select_2_content.text
select_3_title = select_3_title.text
select_3_content = select_3_content.text
txt = txt.text
date, rate, id, skin_type, select_1_title, select_1_content, select_2_title, select_2_content, select_3_title, select_3_content, txt

('2023.07.19',
 '5점만점에 5점',
 '흰모찌이',
 '복합성에 좋아요',
 '피부타입',
 '복합성에 좋아요',
 '피부고민',
 '진정에 좋아요',
 '자극도',
 '자극없이 순해요',
 '\U0001fae0녹는 여름 ...이 더위에 지친 피부를 진정 시키기에\n토너 팩 만한 팩이 없는거 같아요 !!!!\n그래서 이 토너 저 토너 많이 해봤지만\n역시 믿고사는 라운드랩 독도 토너 ....\n\U0001fae0\U0001fae0토너팩으로 해서 피부에 얹어주면\n진정 뽝 됩니다 !!\n\n보습감이 있는 토너는 아니다 보니\n토너팩 하신후 크림으로 마무리 하시면 좋을거 같아요 😘')

### 2. 크롤링 자동화 

In [86]:
page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child(12)')
page.click()

In [95]:
current_page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > strong')
current_page.text

'7'

In [114]:
# try:
for j in range(1, 11):  
    current_page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > strong')
    if current_page.text == str(10*i+j):     # 현재 페이지와 클릭할 페이지 번호가 동일하면 페이지 클릭 없이 그대로 크롤링 진행
        pass    
    else:    # 현재 페이지와 클릭할 페이지 번호가 다르면 페이지 클릭 후 변경된 페이지에서 크롤링 진행
            page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(10*i+j))
            page.click()
            time.sleep(0.5)

In [ ]:
# 해당 제품 내 모든 리뷰 크롤링
# 리뷰 페이지 전환
# 첫 리뷰 페이지에서 1번째 페이지는 클릭이 안된다 => 현재 페이지가 어디인지 확인해서 현재 페이지와 동일하지 않으면 해당 버튼 클릭하는 코드 추가 

for i in range(2): # 제품 내 넘기기 페이지 수 (임의 지정) => 언제까지?
    current_page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > strong')
    for j in range(1, 11):  
            if current_page.text == str(10*i+j):     # 현재 페이지와 클릭할 페이지 번호가 동일하면 크롤링 후 다음페이지 클릭
                page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(10*i+j+1))
            else:    # 현재 페이지와 클릭할 페이지 번호가 다르면 페이지 클릭 후 변경된 페이지에서 크롤링 진행
                    page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a:nth-child({})'.format(10*i+j))
                    page.click()
                    time.sleep(0.5)
        # except:
        #     print(i*10+j)
        #     print('i:', i, 'j:', j)
    next_page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a.next')
    next_page.click()   
    time.sleep(0.5)
# except:
#     print(i, j)

In [88]:
next_page = driver.find_element_by_css_selector('#gdasContentsArea > div > div.pageing > a.next')
next_page.click()

### 3. 한 페이지 리뷰 데이터 크롤링 후 데이퍼 프레임 변환 

In [ ]:
# 리뷰 데이터 크롤링 
# 재구매, 한달사용후기 제외하고 진행 
name = driver.find_element_by_css_selector("#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name").text
brand = driver.find_element_by_css_selector("#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_brand").text
price = int(driver.find_element_by_css_selector("#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike").text.replace(",", ""))
sale_price = int(driver.find_element_by_css_selector("#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong").text.replace(",", ""))
date = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.score_area > span.date').text
rate = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.score_area > span.review_point > span').text
id = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.info > div > p.info_user > a.id').text
skin_type = driver.find_element_by_css_selector('#gdasList > li:nth-child(2) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
purchase_type_2 = driver.find_element_by_css_selector('#gdasList > li:nth-child(9) > div.info > div > div > div:nth-child(1) > a').text
select_1_title = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(1) > dt > span').text
select_1_content = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
select_2_title = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(2) > dt > span').text
select_2_content = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
select_3_title = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(3) > dt > span').text
select_3_content = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text
txt = driver.find_element_by_css_selector('#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner').text

In [ ]:
#gdasList > li:nth-child(1) > div.review_cont > div.txt_inner

#gdasList > li:nth-child(1)